# **Assignment #3**
Due date: **20 Mar 2025 (Tuesday)**

Instructor: Lorenzo Garlappi (lorenzo.garlappi@sauder.ubc.ca)

TA: Siyang Liu (lsy9911@student.ubc.ca), Tina Tian (tina.tian@sauder.ubc.ca)


### Instructions
a. Save this Jupyter notebook under a file name containing your Team number, e.g., `Assignment_3_Team_01.ipynb`, and submit it with your answers through Canvas

b. Enter here below the names and section of each Team members:

1. Name......., Section.....; **Contributed work to the solution of this assignment (in %):** .....  

2. Name......., Section.....; **Contributed work to the solution of this assignment (in %):** .....

3. Name......., Section.....; **Contributed work to the solution of this assignment (in %):** .....

### **Background**

In this assignment, we apply the Chen, Roll, and Ross (1986) model to Blackrock iShare ETF fund returns, implement Mean-Variance approach to calculating the optimal portfolio weights allocated to each fund, and calculate the variance contribution of each risk factor to the total portfolio variance.

This assignment is divided into four parts:

- Part 0 loads data from WRDS and applies Fama-MacBeth regression to estimate risk premiums for Chen, Roll, and Ross (1986) risk factors.

- Part I estimates beta exposure to Chen, Roll, and Ross (1986) risk factors and calculates the expected fund return and covariance matrix. (15 marks)

- Part II applies the Mean-Variance approach to determining the optimal weights allocated to each fund in the following two scenarios: (10 marks)
    - unconstrained optimization, i.e., the minizmized volatility optimization
    - constrained optimization with target returns

- Part III applies the Mean-Variance approach to determining the optimal weights allocated to each fund in the following two scenarios: (40 marks)
    - with target tracking error
    - with target risk appetite

- Part IV maps portfolio returns to Chen, Roll, and Ross (1986) risk factors and calculates the risk factor variance contribution for the optimized portfolio. (35 marks)

### **Part 0: Connect to WRDS - Data Preparation**

This section first connects to WRDS, load stock returns and risk free rates from WRDS. Then we apply Fama-MacBeth regression to Chen, Roll, and Ross (1986) model to estimate the risk premiums for each risk factor.

We fistly check if required packages are installed.

In [4]:
# run this to check if required packages are installed
import subprocess
import sys

def install_packages(packages):
    """
    Ensure that all specified packages are installed. If a package is not installed,
    it will be installed automatically.

    Args:
        packages (list): A list of package names to ensure are installed.

    Returns:
        None
    """
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        else:
            print(f"{package} is already installed.")

required_packages = [
    "numpy",
    "scipy",
    "pandas",
    "wrds",
    "datetime",
    "statsmodels",
    "matplotlib",
    "pathlib",
    "pandas_datareader",
    "sklearn.linear_model",
    "warnings",
    "itertools",
    "joblib",
    "PyPortfolioOpt",
    "seaborn",
    "cvxpy",
    "certifi",
]
install_packages(required_packages)

numpy is already installed.
scipy is already installed.
pandas is already installed.
wrds is already installed.
datetime is already installed.
statsmodels is already installed.
matplotlib is already installed.
pathlib is already installed.
pandas_datareader is already installed.
sklearn.linear_model is already installed.
warnings is already installed.
itertools is already installed.
joblib is already installed.
Installing PyPortfolioOpt...
seaborn is already installed.
cvxpy is already installed.
certifi is already installed.


Please enter the path of tools folder that includes main.py if the code is not in Colab environment.
Or alternatively, you can store the tools folder in the same address where this notebook runs and you don't need to re-enter the folder address and can remove the following code.

In [5]:
import sys
def is_colab():
    return "google.colab" in sys.modules

if not is_colab():
    # path_tools = input("Enter the path of tools folder: ")
    # print(f"You entered: {path_tools }")
    path_tools = "/Users/garlappi/Library/CloudStorage/Dropbox/COMM475/2025/Assignments/Assignment_2" #Enter the path where you stored the "tools" folder
    print(f"You entered: {path_tools }")
    sys.path.append(str(path_tools))

We then import required packages.

In [6]:
### The following packages are for user-defined packages ###
# load user-defined packages

import sys
from pathlib import Path
current_dir = Path().resolve()
sys.path.append(str(current_dir))

# load user-defined packages: these packages are for Fama-MacBeth regressions
from tools.main import clean_test_asset_returns
from tools.main import fama_macbeth_timeseries_estimate_beta
from tools.main import fama_macbeth_crosssection_estimate_premium
from tools.main import fama_macbeth_crosssection_premium_stat
from tools.main import fama_macbeth_regression

# from tools.create_size_portfolios import create_size_portfolios

# run this to make sure that the modules used are the most updated version
import importlib
import tools.main
# import tools.create_size_portfolios
importlib.reload(tools.main)
# importlib.reload(tools.create_size_portfolios)

# load package WRDS
import wrds

# load package for dataframe operation
import pandas as pd
import numpy as np
from datetime import datetime

# load package for Fama-MacBeth regression
from statsmodels.regression.rolling import RollingOLS
import statsmodels.formula.api as smf
import statsmodels.api as sm

# mean-variance optimization
import matplotlib.pyplot as plt
from pypfopt import expected_returns, risk_models, expected_returns, plotting, EfficientFrontier
import seaborn as sns
import cvxpy as cp

# !pip install --upgrade certifi
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


# handle warnings in wrds
import warnings
warnings.filterwarnings("ignore")

Then, we enter credentials and connect to WRDS.
In the following code, enter your user name and password to connect to WRDS:

In [7]:
# WRDS connection
wrds_username = ''

try:
    print("Establishing connection to WRDS database...")
    params = {
        'wrds_hostname': wrds.sql.WRDS_POSTGRES_HOST,
        'wrds_port': wrds.sql.WRDS_POSTGRES_PORT,
        'wrds_dbname': wrds.sql.WRDS_POSTGRES_DB,
        'wrds_username': wrds_username,
        'wrds_connect_args': wrds.sql.WRDS_CONNECT_ARGS,
    }

    conn = wrds.Connection(autoconnect=True, **params)
    print("Successfully connected to WRDS database.")
except Exception as e:
    print(f"Failed to connect to WRDS database: {e}")

Establishing connection to WRDS database...
Enter your WRDS username [root]:tina_sauder
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Successfully connected to WRDS database.


Now we have connected to WRDS. We retrieve monthly stock returns from CRSP data table for Fama-MacBeth regression, and risk-free rates from Fama-French factors data table for Sharpe ratio calculation. We merge CRSP stock returns with risk free rates to calculate the excess returns.

In [8]:
####### load monthly stock returns from CRSP, and risk free rates from Fama-French data table #######

# The start and end date align with the sample period of BlackRock iShare ETF funds in Part II - Part IV
start_date = "06/01/2000"
end_date = "12/01/2024"


############################################################
### load monthly stock returns from CRSP ###

# connect to crsp
crsp_monthly = conn.raw_sql( f"""SELECT permno, siccd, mthcaldt, mthret, mthretx, mthcap,ticker
       FROM crsp.msf_v2 as msf
       WHERE msf.mthcaldt BETWEEN '{start_date}' AND '{end_date}'""",date_cols=['mthcaldt'])

# add year and month to crsp monthly
crsp_monthly = (crsp_monthly.assign(year=lambda x: pd.DatetimeIndex(x["mthcaldt"]).year)
                            .assign(month=lambda x: pd.DatetimeIndex(x["mthcaldt"]).month)
                            )

############################################################


############################################################
### load risk free rates from wrds ###
risk_free_rate_monthly = conn.raw_sql(
    f"""SELECT date, rf
    FROM ff.factors_monthly WHERE date BETWEEN '{start_date}' AND '{end_date}'
    """,
    date_cols=['date']
)

# add year, month, and rename mktrf to mkt_excess
risk_free_rate_monthly = (risk_free_rate_monthly.assign(year=lambda x: pd.DatetimeIndex(x["date"]).year)
                                          .assign(month=lambda x: pd.DatetimeIndex(x["date"]).month)
                                          .rename(columns={"rf": "risk_free_rate"}))

############################################################


############################################################
### merge crsp monthly stock returns with risk free rates ###
crsp_ff3_monthly = (
    crsp_monthly.merge(risk_free_rate_monthly, how="left", on=['year', 'month']))

# calculate excess return for each stock
crsp_ff3_monthly['ret_excess'] = crsp_ff3_monthly['mthret'] - crsp_ff3_monthly['risk_free_rate']

# rename permno to be 'test_id' for Fama-MacBeth regression
crsp_ff3_monthly.rename(columns={'permno':'test_id'}, inplace = True)

# take a look at the data after merge
crsp_ff3_monthly.head(3)

############################################################


,test_id,siccd,mthcaldt,mthret,mthretx,mthcap,ticker,year,month,date,risk_free_rate,ret_excess
0,10001,4925,2000-06-30,0.027607,0.011858,19800.0,EWST,2000,6,2000-06-01,0.004,0.023607
1,10002,6020,2000-06-30,0.04142,0.03125,88553.44,SABC,2000,6,2000-06-01,0.004,0.03742
2,10009,6030,2000-06-30,0.002876,0.002876,75410.06,IROQ,2000,6,2000-06-01,0.004,-0.001124


Next we implement Fama-MacBeth method to Chen, Roll, and Ross (1986) to calculate the risk premiums of the risk factors. The test assets for Fama-MacBeth regression method are 20 size portfolios.

The following code defines test assets and loads Chen, Roll, and Ross (1986) risk factors.

In [9]:
#####################################################################################################
### define a function to create N size portfolio - used for Fama-MacBeth regression test assets ###
def create_size_portfolios(returns_data, N):

    """
    returns_data include:
    - stock id: test_id
    - market cap: mthcap
    - return: mthret
    - date: mthcaldt or date

    N: the number of portfolios to be created

    function output:
    - portfolio returns per date, portfolio
    (1 - N: from the smallest size to the largest size)
    """

    returns_data = returns_data.sort_values(by=['test_id', 'date'])

    # create a lag variable of market value
    returns_data['mthcap_lag'] = returns_data.groupby('test_id')['mthcap'].shift(1)
    returns_data = returns_data.dropna(subset=['mthcap_lag']) # drop if mthcap_lag is NaN

    # assign portfolios based on market cap of previous period (month)
    num_portfolios = N
    size_labels = range(1, N+1) # create a portolio lables

    # assign size portfolios based on lag market cap for each date
    returns_data['portfolio'] = returns_data.groupby('date')['mthcap_lag'].transform(
      lambda x: pd.qcut(x, q=num_portfolios, labels=size_labels)
    )

    # Calculate equally weighted returns for each portfolio, date
    returns_data['e_w_port_size_ret'] = returns_data.groupby(['date','portfolio'])['mthret'].transform('mean')

    # Aggregate portfolio returns per date, portfolio
    portfolio_returns = (
    returns_data[['date', 'portfolio', 'e_w_port_size_ret']]
    .sort_values(by=['date', 'portfolio'])  # Sort by date and portfolio
    .drop_duplicates()  # Drop duplicate rows
    .assign(year=lambda x: pd.DatetimeIndex(x["date"]).year)
    .assign(month=lambda x: pd.DatetimeIndex(x["date"]).month)
    .reset_index(drop=True)  # Reset index for clean output
    )

    return portfolio_returns


#####################################################################
### define test assets - test assets are size portfolios  ###
returns_data = crsp_ff3_monthly
N = 20 # you can design the portfolio size here by seting N

# define test assets as 20 size portfolio
test_assets = create_size_portfolios(returns_data, N)

# merge test assets with Fama-French risk factors and calculate the excess returns for each portfolio
test_assets = test_assets.merge(risk_free_rate_monthly.drop(columns=['date']), how="left", on=['year','month'])
test_assets['ret_excess'] = test_assets['e_w_port_size_ret'] - test_assets['risk_free_rate']

# format the columns names for test_assets to prepare for Fama-MacBeth regression
test_assets = test_assets.rename(columns={'portfolio': 'test_id'}) # rename portfolio as test_id


#####################################################################
### load CRR1986 risk factors  ###
csv_usrec = 'https://raw.githubusercontent.com/lorenzogarlappi/COMM475/refs/heads/main/Data/replicated_UI_DEI_MP_UPR_UTS.csv'
crr_data = pd.read_csv(csv_usrec)

# format risk factors
risk_factors = crr_data[['date','UI','DEI','MP','UPR','UTS']]
risk_factors = risk_factors.dropna()
risk_factors = risk_factors.reset_index(drop = True)

We apply Fama-MacBeth regression to calculate the risk premiums for Chen, Roll, and Ross (1986) risk factors.

In [10]:
risk_premiums = fama_macbeth_regression(test_assets,risk_factors)
risk_premiums

,factor,risk_premium,t_statistic,t_statistic_newey_west
0,Intercept,0.980,2.342,2.255
1,DEI,-0.051,-1.919,-1.702
2,MP,-0.269,-1.662,-1.620
3,UI,-0.079,-1.145,-0.954
4,UPR,0.045,0.743,0.821
5,UTS,0.079,0.331,0.428


### **Part I. Calculate the fund beta exposure to Chen, Roll, and Ross (1986) model and calculate expected fund returns and covariance matrix (15 marks)**


In this section, you need to calculate fund beta exposure to Chen, Roll, and Ross (1986) risk factors ($\beta_{i,UI},\beta_{i,DEI}, \beta_{i,MP}, \beta_{i,UPR}, \beta_{i,UTS} $) and calculate expected fund returns $E[R_i]$ for each fund $i$, and calculate the covariance matrix of the ETF funds.

The risk premiums ($\lambda_{UI}, \lambda_{DEI}, \lambda_{MP}, \lambda_{UPR}, \lambda_{UTS}$) are estimated with Fama-MacBeth regression in Part 0.

$$E[R_i] = r_f + \beta_{i,UI} \lambda_{UI} + \beta_{i,DEI}\lambda_{DEI} + \beta_{i,MP}\lambda_{MP} + \beta_{i,UPR} \lambda_{UPR} + \beta_{i,UTS}\lambda_{UTS} $$

We consider the following 5 BlackRock iShare ETF funds. The sample period is from June 2000 to Nov 2024. Some funds may have shorter sample periods than others.

- **Cash alternative**:

    The iShares Short Treasury Bond ETF seeks to track the investment results of an index composed of U.S. Treasury bonds with remaining maturities one year or less.

    https://www.blackrock.com/us/individual/products/239466/ishares-short-treasury-bond-etf

- **Equity**:

    The iShares Core S&P Total U.S. Stock Market ETF seeks to track the investment results of a broad-based index composed of U.S. equities.

    https://www.blackrock.com/us/individual/products/239724/ishares-core-sp-total-us-stock-market-etf

- **Bond**:

    The iShares U.S. Treasury Bond ETF seeks to track the investment results of an index composed of U.S. Treasury bonds.

    https://www.blackrock.com/us/individual/products/239468/ishares-us-treasury-bond-etf


- **Real estate**:

    The iShares U.S. Real Estate ETF seeks to track the investment results of an index composed of U.S. equities in the real estate sector.

    https://www.ishares.com/us/products/239520/ishares-us-real-estate-etf

- **Inflation**:

    The iShares TIPS Bond ETF seeks to track the investment results of an index composed of inflation-protected U.S. Treasury bonds.

    https://www.ishares.com/us/products/239467/ishares-tips-bond-etf


The following code loads ETF fund returns data; standarizes the BlackRock ETF fund returns; merges the ETF fund return data with Chen, Roll, and Ross (1986) risk factors and risk free rates; and calculates the fund excess returns.

In [11]:
##################      load fund data   ##################
csv_etf_funds = 'https://raw.githubusercontent.com/lorenzogarlappi/COMM475/refs/heads/main/Data/BlackRock_iShares%20funds_consolidated_2000-2025.csv'
etf_funds = pd.read_csv(csv_etf_funds)
etf_funds.head()


##################  format fund return data ##################
# fund returns are transformed to absolute value
etf_funds = (etf_funds.assign(date=lambda x: pd.to_datetime(etf_funds['Month End Date'], errors='coerce'))
                    .rename(columns={"iShares Short Treasury Bond ETF": "cash"})
                    .rename(columns={"iShares Core S&P Total U.S. Stock Market ETF": "equity"})
                    .rename(columns={"iShares U.S. Treasury Bond ETF": "bond"})
                    .rename(columns={"iShares U.S. Real Estate ETF": "real estate"})
                    .rename(columns={"iShares TIPS Bond ETF": "inflation"})
                    .assign(cash=lambda x: pd.to_numeric(x['cash'], errors='coerce') / 100)
                    .assign(equity=lambda x: pd.to_numeric(x['equity'], errors='coerce') / 100)
                    .assign(bond=lambda x: pd.to_numeric(x['bond'], errors='coerce') / 100)
                    .assign(**{'real estate':lambda x: pd.to_numeric(x["real estate"], errors='coerce') / 100})
                    .assign(inflation=lambda x: pd.to_numeric(x['inflation'], errors='coerce') / 100)
                    .assign(date_yyyymm= lambda x: x['date'].dt.strftime('%Y%m'))
                    .drop('Month End Date', axis=1)
                    .set_index('date')
                    .reset_index()
                    )

################## merge etf funds with Chen, Roll, and Ross (1986) risk factors ##################
etf_funds = etf_funds.merge(risk_factors.drop(columns = ['date']), on = ['date_yyyymm'], how = 'left')

################## merge etf funds with risk free rate (for Sharpe ratio calculation) ##################
risk_free_rate_monthly['date_yyyymm'] = risk_free_rate_monthly['date'].dt.strftime('%Y%m')
etf_funds = (etf_funds.merge(risk_free_rate_monthly.drop(columns =['date']), how = "left", on = "date_yyyymm"))

################## calculate fund excess returns ##################
# fund excess retruns = fund return - risk free rate

etf_funds['cash_excess_ret'] = etf_funds['cash'] - etf_funds['risk_free_rate']
etf_funds['equity_excess_ret'] = etf_funds['equity'] - etf_funds['risk_free_rate']
etf_funds['bond_excess_ret'] = etf_funds['bond'] - etf_funds['risk_free_rate']
etf_funds['real_estate_excess_ret'] = etf_funds['real estate'] - etf_funds['risk_free_rate']
etf_funds['inflation_excess_ret'] = etf_funds['inflation'] - etf_funds['risk_free_rate']

etf_funds.head(3)

,date,cash,equity,bond,real estate,inflation,date_yyyymm,UI,DEI,MP,UPR,UTS,risk_free_rate,year,month,cash_excess_ret,equity_excess_ret,bond_excess_ret,real_estate_excess_ret,inflation_excess_ret
0,2000-06-30,NaN,NaN,NaN,NaN,NaN,200006,0.004023,0.001912,0.000753,0.0081,0.0011,0.004,2000,6,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000-07-31,NaN,NaN,NaN,0.0956,NaN,200007,-0.000814,0.000041,-0.001741,0.0070,0.0012,0.0048,2000,7,<NA>,<NA>,<NA>,0.0908,<NA>
2,2000-08-31,NaN,NaN,NaN,-0.0248,NaN,200008,-0.003754,-0.000750,-0.002681,0.0071,-0.0016,0.005,2000,8,<NA>,<NA>,<NA>,-0.0298,<NA>


**Question 1. Regress fund excess return on Chen, Roll, and Ross (1986) risk factors and calculate the beta exposure ($\beta_{i,UI},\beta_{i,DEI}, \beta_{i,MP}, \beta_{i,UPR}, \beta_{i,UTS} $) for each fund $i$. (5 marks)**

**Question 2. Calculate the expected return $E[R_{i}]$ based on the beta exposure to Chen, Roll, and Ross (1986). (5 marks)**

$$E[R_i] = r_{f} + \beta_{i,UI} \lambda_{UI} + \beta_{i,DEI}\lambda_{DEI} + \beta_{i,MP}\lambda_{MP} + \beta_{i,UPR} \lambda_{UPR} + \beta_{i,UTS}\lambda_{UTS} $$

**Question 3. Calculate the covariance matrix for the 5 ETF funds. (5 marks)**

Because the magnitude of the Chen, Roll, and Ross (1986) risk factors covariance matrix are very small ($10^{-6}$ level). You can normalize the risk factor covariance matrix by its Frobenius norm. This will facilitates the mean-variance optimization in Part II.

The following two functions calculate the fund beta exposure to Chen, Roll, and Ross (1986) risk factors, and calculate the expected fund return.

In [12]:
###### this function defines the multiple regression to estimate fund beta exposure ######
def multiple_regression(X,Y):

  """ Args: X: risk factors, Y: fund excess return
      Outputs: regression results
  """

  # Add a constant to X
  X = sm.add_constant(X)

  # Replace NaN values with the mean of the column
  X = X.fillna(X.mean())
  Y = Y.fillna(Y.mean())

  # Fit and summarize OLS model
  results = sm.OLS(Y, X).fit()
  return results


###### this function calculate the expected fund return ######

# risk free rate is the mean risk free rate in the sample period.
risk_free_rate = etf_funds['risk_free_rate'].mean()

# define a function to calculate expected fund return
def cal_fund_expected_return(fund_beta, factor_risk_premium):

  """ Args: fund beta: fund beta exposure to risk factors
            factor risk premium: risk factor premiums estimated with Fama-MacBeth regression
      Outputs: fund expected return
  """

  risk_factors = ['UI', 'DEI', 'MP', 'UPR','UTS']

  fund_returns = 0
  for factor in risk_factors:
    fund_return = fund_beta[factor] * risk_premiums.loc[risk_premiums['factor'] == factor, 'risk_premium'].values[0]
    fund_returns = fund_returns + fund_return

  fund_returns = fund_returns + risk_free_rate

  return fund_returns.round(6)

The following is an example of calculating the beta exposure for cash fund.

In [13]:
# Reference code to calculate beta exposure for cash fund

cash_excess_ret = etf_funds[['UI', 'DEI', 'MP', 'UPR', 'UTS','cash_excess_ret']].dropna() #drop if the fund excess return or CRR risk factors are NAN.
cash_excess_ret = cash_excess_ret[~cash_excess_ret.isin(['--']).any(axis=1)] #drop if the fund excess return or CRR risk factors are NAN.
X = cash_excess_ret[['UI', 'DEI', 'MP', 'UPR','UTS']]
y = cash_excess_ret['cash_excess_ret'].astype('float')
model = multiple_regression(X,y)
cash_beta = model.params.drop(index = 'const')

print(cash_beta)

UI     0.004885
DEI   -0.106855
MP    -0.000359
UPR    0.031612
UTS   -0.000826
dtype: float64


The following is an example of calculating the expected return for cash fund.

In [14]:
# Reference code to calculate the expected return for cash fund
cash_expected_return = cal_fund_expected_return(cash_beta, risk_premiums)
print(cash_expected_return)

0.007931


The following lists the steps to calculate the fund covariance matrix:

(1). Define coefficient matrix of the 5 ETF funds, which are the beta exposure you calculated in Question 1.

(2). Calculate the covariance matrix of the risk factor and normalize the risk factor covariance matrix by the Frobenius norm. The Frobenius norm is defined as the square root of the sum of the absolute squares of its elements.

(3). Calculate the covariance matrix of the 5 ETF funds.

In [15]:
# Reference code to calculate the covariance matrix for the 5 ETF funds.

# for example, we randomly generate the the beta exposure for each fund.
# note: you need update this to the actual beta you calculate in the Question 1
cash_beta = pd.Series(np.random.randn(5))  # Mean=0, Std=1
equity_beta = pd.Series(np.random.randn(5))  # Mean=0, Std=1
bond_beta = pd.Series(np.random.randn(5))  # Mean=0, Std=1
real_estate_beta = pd.Series(np.random.randn(5))  # Mean=0, Std=1
inflation_beta = pd.Series(np.random.randn(5))  # Mean=0, Std=1

### Step 1. Define the coefficient matrix (Shape: (N etf assets x 5 risk factors)): fund exposure to CRR risk factors ###
coef_matrix = np.array([cash_beta,  # exposure to CRR risk factors for cash fund
                        equity_beta, # exposure to CRR risk factors for equity fund
                        bond_beta, # exposure to CRR risk factors for bond fund
                        real_estate_beta, # exposure to CRR risk factors for real estate fund
                        inflation_beta, # exposure to CRR risk factors for inflation fund
])

### Step 2. Calculate the covariance matrix of the CRR risk factor and normalize the risk factor by its Frobenius norm  ###
# CRR 1986 risk factors
X = etf_funds[['UI', 'DEI', 'MP', 'UPR','UTS']]

# Define a function to normalize a covariance matrix by the Frobenius norm, i.e. we scale the entire covariance matrix by its Frobenius norm to make it unit-scaled.
def normalize_covariance_by_frobenius(cov_matrix):
    frob_norm = np.linalg.norm(cov_matrix, 'fro')  # Frobenius norm
    return cov_matrix / frob_norm

# the covariance matrix of CRR 1986 risk factors
cov_risk_factor = X.cov()
# normalize the risk factor covariance matrix by the Frobenius norm
normalized_cov_factor = normalize_covariance_by_frobenius(cov_risk_factor)

### Step 3. Calculate covariance matrix of etf funds: fund exposure to CRR risk factors x normalized covariance matrix of risk factors x fund exposure to CRR risk factors
cov_matrix_funds = coef_matrix @ normalized_cov_factor @ coef_matrix.T
cov_matrix_funds = cov_matrix_funds.to_numpy()

print(cov_matrix_funds.round(6))

[[ 0.088145 -0.044892 -0.073578 -0.044773  0.040205]
 [-0.044892  3.843068 -1.761232 -0.394569  2.296148]
 [-0.073578 -1.761232  1.26563   0.402755 -1.054311]
 [-0.044773 -0.394569  0.402755  0.150269 -0.235882]
 [ 0.040205  2.296148 -1.054311 -0.235882  1.44667 ]]


### **Part II. Apply mean-variance approach to determining the optimal portfolio weights allocated to each fund - unconstrained optimization and optimization with target returns (10 marks)**

In this section, you need to apply Mean-Variance approach to determining the optimal portfolio weights in the following two scenarios:

- unconstrained optimization, i.e., the minizmized volatility portfolio optimization
- constrained optimization with target returns.

The optimization techniques used in this section are the same as what you have done in Assignment #1.

**Question 4. Apply mean-variance approach to determining the optimal portfolio weights of the minimized volatility portfolio; calculate the expected return, total volatility, and Sharpe ratio for the minimized volatility portfolio.** (5 marks)

**Question 5. With a target return of 5% , apply mean-variance approach to determining the optimal portfolio weights. Calculate the expected return, total volatility, and Sharpe ratio for the optimized portfolio.** (5 marks)



Note, if you use Problem Optimization method in cvxpy package, you need to transfrom the fund returns data to series format because the Problem Optimization method in cvxpy package requires returns data to be series.

### **Part III. Apply Mean-Variance approach to determining the optimal portfolio weights - optimization with constraints (tracking error, risk appetite) (40 marks)**

In this section, you need to apply mean-variance approach to optimizing portfolios while incorporating a set of constraints. For each constraint, you nedd to calculate the optimized portfolio's performance as the constraint level varies.

The following two constrains are considered:

- **Target tracking error (TE)**

    Tracking error measures how closely a portfolio follows its benchmark index. It is defined as the standard deviation of the difference between the portfolio returns and the benchmark returns over a given period.

    Benchmark portfolio is the portfolio with 60% invested in equity and 40% invested in bonds.

- **Target risk appetite (or Target total volatility)**

    Risk appetite measures how much risk an investor is willing to take. It is defined as the total volatility of the portfolio.


For each constraint, you need to compare the following performance metrics for the optimized portfolio while the constraint level varies:
- **Total returns**

    The expected portfoio returns

- **Total risk**

    The portfolio volatility

- **Sharpe ratio**

    The portfolio excess returns divided by the portfolio volatility

- **TE. vs BM**

    Tracking error of the portfolio against benchmark portfolio


**Question 6. Apply mean-variance approach to fund returns while incorporating tracking error constraints. Compare the portfolio performance metrics among benchmark portfolio, optimized portfolio with target tracking error 0.02, optimized portfolio with target tracking error 0.06.** (20 marks)

Ideally, you would like to have a table to compare the portfolio performance metrics as the tracking error constraint varies:

| **Metric**            | **BM** | **0.02 TE** | **0.06 TE** |
|-----------------------|--------|-------------|-------------|
| **Total Return**      | -      | -           | -           |
| **Total Risk**        | -      | -           | -           |
| **Sharpe Ratio**      | -      | -           | -           |
| **Tracking Error vs BM** | -      | -           | -           |



**Question 7. Apply mean-variance approach to fund returns while incorporating risk appetite constraints. Compare the portfolio performance among optimized portfolio with target risk appetite 0.006, optimized portfolio with target risk appetite 0.008, optimized portfolio with target risk appetite 0.012.** (20 marks)

Ideally, you would like to have a table to compare the portfolio performance metrics as the risk appetite constraint varies:

| **Metric**            | **0.006 risk appetite** | **0.008 risk appetite** | **0.012 risk appetite** |
|-----------------------|--------|-------------|-------------|
| **Total Return**      | -      | -           | -           |
| **Total Risk**        | -      | -           | -           |
| **Sharpe Ratio**      | -      | -           | -           |
| **Tracking Error vs BM** | -      | -           | -           |

The following reference code defines the benchmark portfolios, and calculate the performance of the benchmark portfolio.

In [16]:
### Benchmark portfolio ###


# initiate a dictionary to define the performance metrix for benchmark portfolio
portfolio_comp_te={'BM':''}
# define the performance metrics
portfolio_stat ={'Tot ret':'','Tot risk':'','Sharpe ratio':'','TE. vs BM':''}

# defin risk free rate
risk_free_rate = etf_funds['risk_free_rate'].mean()

# define the benchmark: 60% equity + 40% bond
bm_weights = [0,0.6,0.4,0,0]
bm_weights = np.array(bm_weights).reshape(1, -1)

# define fund expected returns
# note: you need update this to be actual fund expected returns you calculated in Question 2.
series_fund_expected_returns = pd.Series(np.random.randn(5))

# calculate the expected return, volatility, sharp ratio, and tracking error
cov_matrix_port_bm = bm_weights @ cov_matrix_funds @ bm_weights.T # variance for benchmark portfolio
expected_ret_port_bm = bm_weights @ series_fund_expected_returns  # expected return for benchmark portfolio
sharpe_ratio_port_bm = (expected_ret_port_bm - risk_free_rate) / cov_matrix_port_bm # sharpe ratio for benchmark portfolio
te_ratio_port_bm = 0 # tracking error for benchmark portfolio

portfolio_stat['Tot ret'] = expected_ret_port_bm[0]
portfolio_stat['Tot risk'] = cov_matrix_port_bm[0][0]
portfolio_stat['Sharpe ratio'] = sharpe_ratio_port_bm[0][0]
portfolio_stat['TE. vs BM'] = te_ratio_port_bm

portfolio_comp_te['BM'] = portfolio_stat

# transform the dictionary to a dataframe
pd.DataFrame(portfolio_comp_te).round(6)


,BM
Tot ret,-0.301830
Tot risk,0.740614
Sharpe ratio,-0.409449
TE. vs BM,0.000000


The following reference code calculates the tracking error, and optimizes portfolio with targed tracking error.

In [17]:
### Reference code ###

### calculate the optimal portfolio weights for a given target tracking error ###

fund_names = ['cash', 'equity', 'bond', 'real estate', 'inflation']

risk_free_rate = etf_funds['risk_free_rate'].mean()

target_te = 0.01

# define the expected returns and covariance matrix
frequency = 1 #monthly return
mu = series_fund_expected_returns # expected returns
S = cov_matrix_funds # estimated covariance matrix
w = cp.Variable(len(mu)) # weights to optimize

# calculate the tracking error squared
te_squared = cp.mean((w @ mu - expected_ret_port_bm[0]) ** 2)

# optimize the expected returns subject to wT1=1 and target tracking error constraint
problem = cp.Problem(
    objective=cp.Minimize(w @ S @ w),
    constraints=[
    w >= 0,
    cp.sum(w) == 1,
    te_squared <= target_te**2,
    ],
)
problem.solve()
weights = pd.Series(w.value,fund_names,name='weight')

######### calculate the portfolio performance #########
# portfolio expected returns
expected_return = weights.ravel() @ mu.ravel()
# portfolio volatility
volatility = np.sqrt(weights @ S @ weights)
# portfolio sharpe ratio
sharpe_ratio = (expected_return - risk_free_rate) / volatility
# portfolio tracking error
te = np.sqrt(np.mean((weights.ravel() @ mu.ravel() - expected_ret_port_bm[0]) ** 2))

### **Part IV. Mapping optimized portfolio to Chen, Roll, and Ross (1986) risk factors and calculate the variance contribution of risk factors to the total portfolio variance. (35 marks)**

In this section, firstly you need to apply mean-variance approach to determining the optimal portfolio weights with a target return of 5% (similar to what you have done in Part II).

Then you need to map the optimized portfoio returns to Chen, Roll, and Ross (1986) risk factors. Calculate the portfolio exposure to Chen, Roll, and Ross (1986) risk factors, and calculate the variance contribution of each risk factor to the optimized portfolio variance.

Steps to understand the variance decomposition method:

1. The optimized portfoio return is a weighted average of the 5 ETF fund returns

$$𝑅_{p}=\sum_{i=1}^{5}\omega_{i}R_{i} $$

where:

$\omega_{𝑖}$: the optimal weight allocated to fund $𝑖$.

2. For each ETF fund $i$, the fund return ($𝑅_{i}$) is explained by Chen, Roll, and Ross (1986) 5 risk factors

$$𝑅_{i}=\beta_{i,1}*F_{1} + \beta_{i,2}*F_{2} + ... +\beta_{i,5}*F_{5} + \epsilon_{i}$$

Where:

$\beta_{𝑖,1},...,\beta_{𝑖,5}$: Sensitivity of the fund $i$ to risk factor $𝐹_{1},...,𝐹_{5}$.

$\epsilon_{i}$: Unexplained (idiosyncratic) component.


3. Calculate the total variance of portfolio return

$$Var(𝑅_{p})=\sum_{i=1}^{5}
\omega_{i}^{2}Var(R_{i}) + 2\sum_{i<k}\omega_{i}\omega_{k}Cov(R_{i},R_{k})$$

where for each $R_{i}$:

$$Var(𝑅_{i})={\sum_{k=1}^{5}
\beta_{i,k}^{2}Var(F_{k}) + 2\sum_{m<n}\beta_{i,m}\beta_{i,n}Cov(𝐹_{m},𝐹_{n})}+Var(\epsilon_{i})$$

4. Calculate the variance contribution of risk factor $F_{k}$

- Direct contribution:

  - $\sum_{i=1}^{5}\omega_{i}^{2}\beta_{𝑖,k}^{2}Var(𝐹_{k})$, variance contribution to portfolio risk by risk factor $𝐹_{k}$.

- The normalized variance contribution of risk factor $𝐹_{k}$ to the total portfolio variance is:

$$Contribution Factor_{k}=\frac{\sum_{i=1}^{5}\omega_{i}^{2}\beta_{𝑖,k}^{2}Var(𝐹_{k})}{Var(𝑅_{𝑝})}$$

Therefore, the portfolio variance attributed to a risk factor $k$ ($Contribution Factor_{k}$) is determined by:

- the portfolio weights of fund $i$:  $\omega_{i}$  
- the fund $i$ risk exposure to risk factors $k$:  $\beta_{i,k}$
- the covariance matrix of risk factors: $Var(F_{k})$

**Question 8. Apply mean-variance approach to determining the optimal fund weights that minimize portfolio volatility with a target return of 5%. For the optimized portfolio, calculate the portfolio exposure to Chen, Roll, and Ross (1986) risk factors. (15 marks)**

Mathematically, the portfolio exposure to Chen, Roll, and Ross (1986) risk factor $k$ is:

$$\beta_{p,k} = \sum_{i=1}^{5}\omega_{i}\beta_{i,k}$$

**Question 9. For the minimized volatility portfolio, calculate the variance contribution of each risk factor ($Contribution Factor_{k}$). Which risk factor has the highest variance contribution and which risk factor has the lowest variance contribution ? (20 marks)**